# SKBL -> Kepler.gl 
version 0.2 
* This [Notebook](https://github.com/salgo60/FactGrid/blob/master/SKBL -> Kepler.gl.gl.ipynb)  
  * the vision is to create something like [this](https://database.factgrid.de/tools/illuminati.html) in this Notebook but for SKBL
  * [video](https://www.youtube.com/watch?v=WfRQQJ0vO44) showing how binder is started and used


OBS right now you need to **install [Jupiter Notebook](https://jupyter.org/install.html)** on your local machine to run Kepler.gl we have not found a way of getting it work using e.g. [binder](https://mybinder.org/). The sparql -> [pandas](https://pandas.pydata.org/) part works using [binder](https://mybinder.org/) but not [Kepler.gl](https://pandas.pydata.org/).... 

### Misc links

* [The Illuminati Correspondence Fast Forward](https://blog.factgrid.de/archives/1695) the blogpost we should try to do in Jupyter
* [Create a Quick Web Map with Kepler.gl and Jupyter Notebook](https://spatial.blog.ryerson.ca/2019/11/18/create-quick-dashboard-with-kepler-gl-and-jupyter-notebook/)
* Video [Visualization Nights - Introduction to Kepler.gl](https://www.youtube.com/watch?v=b8wKEY4dlvg)
* [From Beautiful Maps to Actionable Insights: Introducing kepler.gl, Uber’s Open Source Geospatial ToolboxThe](https://eng.uber.com/keplergl/)
* [Kepler.GL & Jupyter Notebooks: Geospatial Data Visualization with Uber’s opensource Kepler.GL](https://towardsdatascience.com/kepler-gl-jupyter-notebooks-geospatial-data-visualization-with-ubers-opensource-kepler-gl-b1c2423d066f)
* [Introducing kepler.gl for Jupyter](https://medium.com/vis-gl/introducing-kepler-gl-for-jupyter-f72d41659fbf)
* [Learn How to Visualize Geospatial Data in Jupyter using kepler.gl](https://www.analyticsvidhya.com/blog/2020/06/learn-visualize-geospatial-data-jupyter-kepler/)  
* New Python utility package [wikidata2df](https://github.com/jvfe/wikidata2df)


In [1]:
import json,sys
import pandas as pd 
from SPARQLWrapper import SPARQLWrapper, JSON
endpointWD_url = "https://query.wikidata.org/sparql"

def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

1. Get the SPARQL

In [2]:
querySKBL= """SELECT ?item ?itemLabel ?date ?origin ?originLabel 
 ?origin_latitude ?origin_longitude ?destination ?destinationLabel 
  ?destination_latitude ?destination_longitude 
  ?SKBL WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
  ?item wdt:P4963 ?SKBLid. 
  BIND(URI(CONCAT("https://www.skbl.se/sv/artikel/",?SKBLid)) AS ?SKBL) 
  ?item wdt:P569 ?date.
  ?item wdt:P19 ?origin.
  ?origin p:P625 [
          psv:P625 [
             wikibase:geoLatitude ?origin_latitude ;
             wikibase:geoLongitude ?origin_longitude 
           ] 
         ].  
  ?item wdt:P20 ?destination.
  ?destination p:P625 [
          psv:P625 [
             wikibase:geoLatitude ?destination_latitude ;
             wikibase:geoLongitude ?destination_longitude 
           ] 
         ]
}"""


2. Run the SPARQL

In [3]:
resultsSKBL = get_sparql_dataframe(endpointWD_url, querySKBL)


3. Check result

In [4]:
resultsSKBL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1220 entries, 0 to 1219
Data columns (total 12 columns):
item                     1220 non-null object
itemLabel                1220 non-null object
date                     1220 non-null object
origin                   1220 non-null object
originLabel              1220 non-null object
origin_latitude          1220 non-null object
origin_longitude         1220 non-null object
destination              1220 non-null object
destinationLabel         1220 non-null object
destination_latitude     1220 non-null object
destination_longitude    1220 non-null object
SKBL                     1220 non-null object
dtypes: object(12)
memory usage: 114.5+ KB


4. Change data types

In [10]:
resultsSKBL['origin_latitude'] = resultsSKBL[['origin_latitude']].astype(float).values
resultsSKBL['origin_longitude'] = resultsSKBL[['origin_longitude']].astype(float).values
resultsSKBL['destination_latitude'] = resultsSKBL[['destination_latitude']].astype(float).values
resultsSKBL['destination_longitude'] = resultsSKBL[['destination_longitude']].astype(float).values
resultsSKBL.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1220 entries, 0 to 1219
Data columns (total 12 columns):
item                     1220 non-null object
itemLabel                1220 non-null object
date                     1220 non-null object
origin                   1220 non-null object
originLabel              1220 non-null object
origin_latitude          1220 non-null float64
origin_longitude         1220 non-null float64
destination              1220 non-null object
destinationLabel         1220 non-null object
destination_latitude     1220 non-null float64
destination_longitude    1220 non-null float64
SKBL                     1220 non-null object
dtypes: float64(4), object(8)
memory usage: 114.5+ KB


In [11]:
resultsSKBL

,item,itemLabel,date,origin,originLabel,origin_latitude,origin_longitude,destination,destinationLabel,destination_latitude,destination_longitude,SKBL
0,http://www.wikidata.org/entity/Q4949708,Esther Gehlin,1892-03-24T00:00:00Z,http://www.wikidata.org/entity/Q1748,Köpenhamn,55.676111,12.568889,http://www.wikidata.org/entity/Q25411,Helsingborg,56.042412,12.720996,https://www.skbl.se/sv/artikel/EstherGehlin
1,http://www.wikidata.org/entity/Q4956908,Barbro Johansson,1912-09-25T00:00:00Z,http://www.wikidata.org/entity/Q2211,Malmö,55.593189,13.021423,http://www.wikidata.org/entity/Q25286,Uppsala,59.849761,17.638947,https://www.skbl.se/sv/artikel/BarbroJohansson
2,http://www.wikidata.org/entity/Q4933592,Margit Abenius,1899-07-14T00:00:00Z,http://www.wikidata.org/entity/Q935348,Orsa,61.090147,14.628894,http://www.wikidata.org/entity/Q25286,Uppsala,59.849761,17.638947,https://www.skbl.se/sv/artikel/MargitAbenius
3,http://www.wikidata.org/entity/Q18238701,Hilda Ericsson,1860-08-03T00:00:00Z,http://www.wikidata.org/entity/Q10519255,Hedvig Eleonora församling,59.335603,18.081128,http://www.wikidata.org/entity/Q10519255,Hedvig Eleonora församling,59.335603,18.081128,https://www.skbl.se/sv/artikel/HildaEricsson
4,http://www.wikidata.org/entity/Q15695053,Gun Bergman,1916-10-28T00:00:00Z,http://www.wikidata.org/entity/Q25413,Linköping,58.409418,15.625656,http://www.wikidata.org/entity/Q10519255,Hedvig Eleonora församling,59.335603,18.081128,https://www.skbl.se/sv/artikel/GunBergman
...,...,...,...,...,...,...,...,...,...,...,...,...
1215,http://www.wikidata.org/entity/Q4960837,Mathilda Langlet,1832-11-08T00:00:00Z,http://www.wikidata.org/entity/Q10466076,Dalarö församling,59.130556,18.405833,http://www.wikidata.org/entity/Q10674865,Spetebyhall,58.995000,16.416389,https://www.skbl.se/sv/artikel/MathildaLanglet
1216,http://www.wikidata.org/entity/Q96694766,Alfa Olsson,1914-07-24T00:00:00Z,http://www.wikidata.org/entity/Q27455,Kungälv,57.872345,11.975785,http://www.wikidata.org/entity/Q27455,Kungälv,57.872345,11.975785,https://www.skbl.se/sv/artikel/AlfaSofiaOlsson
1217,http://www.wikidata.org/entity/Q4948018,Eddie Figge,1904-02-04T00:00:00Z,http://www.wikidata.org/entity/Q10659387,S:t Görans församling,59.334128,18.027472,http://www.wikidata.org/entity/Q3433770,Adolf Fredriks församling,59.337944,18.060111,https://www.skbl.se/sv/artikel/EddieFigge
1218,http://www.wikidata.org/entity/Q4953537,Elli Hemberg,1896-11-13T00:00:00Z,http://www.wikidata.org/entity/Q58674640,Skövde stadsförsamling,58.389200,13.847200,http://www.wikidata.org/entity/Q3433770,Adolf Fredriks församling,59.337944,18.060111,https://www.skbl.se/sv/artikel/ElliHemberg


5. Load the data to Kepler.gl    

**OBS** this part works only if you run [Jupiter Notebook](https://jupyter.org/install.html) on your local machine

your current map configuration
* about saving the [config in Kepler.gl](https://docs.kepler.gl/docs/keplergl-jupyter#5-save-and-load-config)

In [13]:
#Show data in Kepler.gl map is nor shown in GITHUB 
import geopandas as gpd   
from keplergl import KeplerGl #importing KeplerGl
gdfSKBL = gpd.GeoDataFrame(resultsSKBL, geometry=gpd.points_from_xy(resultsSKBL.origin_longitude, resultsSKBL.origin_latitude))
SKBL = KeplerGl(height=600, width=800)  
SKBL.add_data(data=gdfSKBL, name="SKBL") 
SKBL #show the map not seen when file uploaded to GITHUB 


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'SKBL': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21…

More about config and saving of kepler.gl
* [Save and export](https://docs.kepler.gl/docs/user-guides/k-save-and-export)

More links 
* [How to Create Eye-Catching Maps With Python and Kepler](https://medium.com/nightingale/how-to-create-eye-catching-maps-with-python-and-kepler-gl-e7e897eff8ac#fe6b-d40ec8d73bab)
* The Programming Historian today: [Introduction to Jupyter Notebooks](https://programminghistorian.org/en/lessons/jupyter-notebooks)

In [14]:
gdfSKBL.to_file("SKBL.geojson", driver='GeoJSON')

* see [raw file](https://raw.githubusercontent.com/salgo60/SKBLWikidata/master/web/SKBL.geojson)
* test [websida]